In [2]:
from codelin.models.const_tree import C_Tree
from codelin.encs.constituent import *
from codelin.utils.constants import *
from codelin.models.linearized_tree import LinearizedTree
from codelin.models.const_label import C_Label
from nltk.tree import Tree
from nltk.treetransforms import collapse_unary

import pandas as pd
import os
import re
import copy

def pt(t, d=True):
    '''
    Print tree if d is True
    '''
    if d:
        if type(t) is list:
            for i in t:
                print(i)
            for i in t:
                Tree.fromstring(str(i)).pretty_print()
        else:
            Tree.fromstring(str(t)).pretty_print()

def pb(ts, dir="r"):
    '''
    Print binary tree
    '''
    t = C_Tree.from_string(str(ts))
    t = t.collapse_unary()
    
    if dir == "r":
        t = C_Tree.to_binary_right(t)
    elif dir == "l":
        t = C_Tree.to_binary_left(t)
    else:
        raise ValueError("dir must be 'r' or 'l'")
    
    Tree.fromstring(str(t)).pretty_print()

def debug_print(st, dt, lt):
    print("Error unmatch at")
    print(st)
    print(dt)
    print("With linearization")
    print(lt)


def clean_features(t):
    '''
    remove all characters between ## in all the nodes of the tree
    '''
    regex = re.compile(r'##.*?##')
    label = regex.sub('', t.label)
    children = [clean_features(c) for c in t.children]
    return C_Tree(label, children)

##
# store trees that gave problems 
##

# genia
g01 = "(ROOT (NONE The) (NONE 4-carboranyl-substituted) (NONE compounds) (NONE -LRB-) (NONE 7) (NONE , ) (NONE 11) (NONE -RRB-) (NONE showed) (NONE antagonistic) (NONE activity) (NONE but) (NONE no) (NONE agonistic) (NONE activity) (NONE even) (NONE in) (NONE the) (NONE presence) (NONE of) (NONE the) (NONE potent) (NONE synergist) (NONE HX630) (NONE .))"
g02 = "(ROOT (NONE In) (NONE parallel) (NONE ,) (NONE NAC) (NONE was) (NONE shown) (NONE to) (NONE down-regulate) (NONE the) (NONE production) (NONE of) (protein (NONE cytokines)) (NONE by) (cell_type (NONE DC)) (NONE as) (NONE well) (NONE as) (NONE their) (NONE surface) (NONE expression) (NONE of) (protein (NONE HLA-DR)) (NONE ,) (protein (NONE CD86)) (NONE -LRB-) (protein (NONE B7-2)) (NONE -RRB-) (NONE ,) (NONE and) (protein (NONE CD40) (NONE molecules)) (NONE both) (NONE at) (NONE the) (NONE basal) (NONE state) (NONE and) (NONE upon) (NONE LPS) (NONE activation) (NONE .))"
g03 = "(ROOT (NONE The) (RNA (protein (NONE MNDA)) (NONE mRNA)) (NONE level) (NONE in) (cell_type (NONE primary) (NONE granulocytes)) (NONE was) (NONE unaffected) (NONE by) (NONE addition) (NONE of) (protein (NONE interferon) (NONE alpha)) (NONE and) (NONE other) (NONE agents) (NONE including) (protein (NONE interferon) (NONE gamma)) (NONE ,) (protein (NONE endotoxin)) (NONE ,) (protein (NONE poly-LRB-I-RRB-) (NONE poly-LRB-C-RRB-)) (NONE ,) (NONE and) (protein (NONE FMLP)) (NONE .))"
g04 = "(ROOT (NONE The) (NONE expression) (NONE of) (NONE c-fos) (NONE ,) (NONE c-jun) (NONE and) (NONE jun) (NONE B) (NONE proto-oncogenes) (NONE was) (NONE studied) (NONE in) (cell_line (protein (NONE phytohemagglutinin)) (NONE -LRB-) (protein (NONE PHA)) (NONE -RRB-) (NONE activated) (NONE peripheral) (NONE blood) (NONE lymphocytes)) (NONE -LRB-) (cell_type (NONE PBL)) (NONE -RRB-) (NONE from) (NONE young) (NONE and) (NONE aged) (NONE humans) (NONE .))"
g05 = "(ROOT (cell_line (NONE E2) (NONE -stimulated) (NONE ER+) (NONE cells)) (NONE were) (NONE more) (NONE susceptible) (NONE to) (NONE lysis) (NONE by) (cell_type (NONE LAK) (NONE cells)) (NONE than) (NONE corresponding) (cell_line (NONE TAM) (NONE -treated)) (NONE or) (cell_type (NONE control) (NONE cells)) (NONE ,) (NONE while) (NONE treatment) (NONE of) (cell_line (NONE ER-cells)) (NONE with) (NONE either) (NONE E2) (NONE or) (NONE TAM) (NONE alone) (NONE did) (NONE not) (NONE alter) (NONE from) (NONE control) (NONE their) (NONE susceptibility) (NONE to) (NONE this) (NONE immune-mediated) (NONE lysis) (NONE .))"
# ptb
p01 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
p02 = "(SINV (S (ADVP (RB Once) (RB again)) (-LRB- -LCB-) (NP (DT the) (NNS specialists)) (-RRB- -RCB-) (VP (VBD were) (RB not) (ADJP (JJ able) (S (VP (TO to) (VP (VB handle) (NP (NP (DT the) (NNS imbalances)) (PP (IN on) (NP (NP (DT the) (NN floor)) (PP (IN of) (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)))))))))))) (, ,) ('' '') (VP (VBD said)) (NP (NP (NNP Christopher) (NNP Pedersen)) (, ,) (NP (NP (JJ senior) (NN vice) (NN president)) (PP (IN at) (NP (NNP Twenty-First) (NNP Securities) (NNP Corp))))) (. .))"
p03 = "(S (NP (NP (NNP Seven) (NNP Big) (NNP Board) (NNS stocks)) (: --) (NP (NP (NNP UAL)) (, ,) (NP (NNP AMR)) (, ,) (NP (NNP BankAmerica)) (, ,) (NP (NNP Walt) (NNP Disney)) (, ,) (NP (NNP Capital) (NNP Cities\/ABC)) (, ,) (NP (NNP Philip) (NNP Morris)) (CC and) (NP (NNP Pacific) (NNP Telesis) (NNP Group))) (: --)) (VP (VP (VBD stopped) (S (VP (NN trading)))) (CC and) (VP (ADVP (RB never)) (VBD resumed))) (. .))"
p04 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
p05 = "(FRAG (SBAR (IN As) (IN in) (: :) (`` ``) (SQ (NP (PRP You)) (VP (VBD went) (NP (VBG ballooning))) (. ?) (. ?) (. !) (. !))))"
p06 = "(NP (NP (NN Year)) (VP (VBN ended) (NP (NNP Dec.) (CD 31) (, ,) (CD 1988))) (X (NN !)))"
p07 = "(S (INTJ (RB No)) (, ,) (NP (PRP it)) (VP (VBD was) (RB n't) (NP (NNP Black) (NNP Monday))) (. .))"
p08 = "(S (NP (DT The) (JJ finger-pointing)) (VP (VBZ has) (ADVP (RB already)) (VP (VBN begun))) (. .))"
p09 = "(S (`` ``) (NP (DT The) (NN equity) (NN market)) (VP (VBD was) (ADJP (JJ illiquid))) (. .))"
p10 = "(S (PP (IN In) (NP (NP (DT an) (NNP Oct.) (CD 19) (NN review)) (PP (IN of) (NP (`` ``) (NP (DT The) (NN Misanthrope)) ('' '') (PP (IN at) (NP (NP (NNP Chicago) (POS 's)) (NNP Goodman) (NNP Theatre))))) (PRN (-LRB- -LRB-) (`` ``) (S (NP (JJ Revitalized) (NNS Classics)) (VP (VBP Take) (NP (DT the) (NNP Stage)) (PP (IN in) (NP (NNP Windy) (NNP City))))) (, ,) ('' '') (NP (NNP Leisure) (CC &) (NNP Arts)) (-RRB- -RRB-)))) (, ,) (NP (NP (NP (DT the) (NN role)) (PP (IN of) (NP (NNP Celimene)))) (, ,) (VP (VBN played) (PP (IN by) (NP (NNP Kim) (NNP Cattrall)))) (, ,)) (VP (VBD was) (VP (ADVP (RB mistakenly)) (VBN attributed) (PP (TO to) (NP (NNP Christina) (NNP Haag))))) (. .))"
p11 = "(NP (NNP Conrail))"
p12 = "(S (NP (NP (RB Not) (PDT all) (DT those)) (SBAR (WHNP (WP who)) (S (VP (VBD wrote))))) (VP (VBP oppose) (NP (DT the) (NNS changes))) (. .))"
p13 = "(SINV (S (NP (DT The) (NNS changes)) (VP (VBD were) (VP (VBN proposed) (PP (IN in) (NP (DT an) (NN effort) (S (VP (TO to) (VP (VP (VB streamline) (NP (JJ federal) (NN bureaucracy))) (CC and) (VP (NN boost) (NP (NP (NN compliance)) (PP (IN by) (NP (NP (DT the) (NNS executives)) (`` ``) (SBAR (WHNP (WP who)) (S (VP (VBP are) (ADVP (RB really)) (VP (VBG calling) (NP (DT the) (NNS shots)))))))))))))))))) (, ,) ('' '') (VP (VBD said)) (NP (NP (NNP Brian) (NNP Lane)) (, ,) (NP (NP (JJ special) (NN counsel)) (PP (IN at) (NP (NP (NP (NP (DT the) (NNP SEC) (POS 's)) (NN office)) (PP (IN of) (NP (NN disclosure) (NN policy)))) (, ,) (SBAR (WHNP (WDT which)) (S (VP (VBD proposed) (NP (DT the) (NNS changes))))))))) (. .))"
p14 = "(S (PP (IN In) (NP (NP (DT an) (NNP Oct.) (CD 19) (NN review)) (PP (IN of) (NP (`` ``) (NP (DT The) (NN Misanthrope)) ('' '') (PP (IN at) (NP (NP (NNP Chicago) (POS 's)) (NNP Goodman) (NNP Theatre))))) (PRN (-LRB- -LRB-) (`` ``) (S (NP (JJ Revitalized) (NNS Classics)) (VP (VBP Take) (NP (DT the) (NNP Stage)) (PP (IN in) (NP (NNP Windy) (NNP City))))) (, ,) ('' '') (NP (NNP Leisure) (CC &) (NNP Arts)) (-RRB- -RRB-)))) (, ,) (NP (NP (NP (DT the) (NN role)) (PP (IN of) (NP (NNP Celimene)))) (, ,) (VP (VBN played) (PP (IN by) (NP (NNP Kim) (NNP Cattrall)))) (, ,)) (VP (VBD was) (VP (ADVP (RB mistakenly)) (VBN attributed) (PP (TO to) (NP (NNP Christina) (NNP Haag))))) (. .))"
p15 = "(S (NP (DT The) (NN luxury) (NN auto) (NN maker)) (NP (JJ last) (NN year)) (VP (VBD sold) (NP (CD 1,214) (NNS cars)) (PP (IN in) (NP (DT the) (NNP U.S.)))))"

# spmrl
s01 = "(VROOT (PN (NE-PNC##lem=Grand|case=*|number=*|gender=*## GRAND) (NE-PNC##lem=Rapids|case=nom|number=sg|gender=neut## RAPIDS)) ($,##lem=--|_## ,) (NP (ADJA-NK##lem=3.|case=nom|number=sg|gender=masc|degree=pos## 3.) (NN-NK##lem=Januar|case=nom|number=sg|gender=masc## Januar)) ($LRB##lem=--|_## -LRB-) (NE##lem=afp|case=nom|number=sg|gender=*## afp) ($LRB##lem=--|_## -RRB-) ($.##lem=--|_## .))"
s02 = "(VROOT (S (PP-MO (APPR-AC##lem=in|_## In) (ART-NK##lem=der|case=dat|number=pl|gender=*## den) (NE-NK##lem=USA|case=dat|number=pl|gender=*## USA)) (VAFIN-HD##lem=haben|number=sg|person=3|tense=pres|mood=ind## hat) (PRF-OA##lem=sich|case=acc|number=sg|person=3## sich) (NP-SB (ART-NK##lem=ein|case=nom|number=sg|gender=neut## ein) (AP-NK (NP-AMS (CARD-NK##lem=neun|_## neun) (NN-NK##lem=Jahr|case=acc|number=pl|gender=neut## Jahre)) (ADJA-HD##lem=alt|case=nom|number=sg|gender=neut|degree=pos## altes)) (NN-NK##lem=Mädchen|case=nom|number=sg|gender=neut## Mädchen)) (CVP-OC (VP-CJ (ADJD-MO##lem=erfolgreich|degree=pos## erfolgreich) (PP-MO (APPR-AC##lem=als|_## als) (NN-NK##lem=Geburtshelferin|case=nom|number=sg|gender=fem## Geburtshelferin)) (VVPP-HD##lem=betätigen|_## betätigt)) (KON-CD##lem=und|_## und) (VP-CJ (VVPP-HD##lem=helfen|_## geholfen)))) ($,##lem=--|_## ,) (VP-OC (NP-OA (PPOSAT-NK##lem=sein|case=acc|number=sg|gender=masc## seinen) (ADJA-NK##lem=klein|case=acc|number=sg|gender=masc|degree=pos## kleinen) (NN-NK##lem=Bruder|case=acc|number=sg|gender=masc## Bruder)) (PP-MO (APPRART-AC##lem=zu|case=dat|number=sg|gender=fem## zur) (NN-NK##lem=Welt|case=dat|number=sg|gender=fem## Welt)) (VZ-HD (PTKZU-PM##lem=zu|_## zu) (VVINF-HD##lem=bringen|_## bringen))) ($.##lem=--|_## .))"
s03 = "(NE##lem=USA|case=nom|number=pl|gender=*## USA)"
s04 = "(VROOT (S (NP-SB (ART-NK##lem=der|case=nom|number=sg|gender=neut## Das) (NN-NK##lem=Ministerium|case=nom|number=sg|gender=neut## Ministerium)) (VAFIN-HD##lem=haben|number=sg|person=3|tense=pres|mood=ind## hat) (NP-OA (AVP-MO (ADV-HD##lem=nur|_## nur) (ADV-MO##lem=noch|_## noch)) (ART-NK##lem=der|case=acc|number=sg|gender=fem## die) (NN-NK##lem=Fachaufsicht|case=acc|number=sg|gender=fem## Fachaufsicht))) ($.##lem=--|_## .))"

### Get depth of a tree

In [3]:
from codelin.models.const_tree import C_Tree

t01 = "(ROOT (A a) (B b) (C c))"
t02 = "(ROOT (A a) (B b) (C c) (D d))"
t03 = "(ROOT (A a) (NT1 (B b) (C c) (NT2 (D d) (E e))))"
t04 = "(S (NP (D The) (N child)) (VP (V did) (Neg not) (VP (V try) (to-P (P to) (VP (V eat) (N anything))))))"

t_str = t04
tree = C_Tree.from_string(t_str)
# collapse postags?
tree = tree.collapse_unary("+")
tree = tree.remove_preterminals()
branching = tree.branching()
pt(tree)
print("The tree has branching: ", branching)
print("The tree has a {} % of left branching".format(branching["L"]))
print("The tree has a {} % of right branching".format(branching["R"]))

                   S                               
      _____________|___                             
     |                 VP                          
     |          _______|_______                     
     |         |   |           VP                  
     |         |   |    _______|___                 
     |         |   |   |          to-P             
     |         |   |   |    _______|____            
     NP        |   |   |   |            VP         
  ___|____     |   |   |   |        ____|_____      
The     child did not try  to     eat      anything

The tree has branching:  {'L': 20.0, 'R': 80.0}
The tree has a 20.0 % of left branching
The tree has a 80.0 % of right branching


### Some encodings

In [3]:
# lin_tree
# l01 = "-BOS-	-BOS-	-BOS-\n\
# Ms.	NNP	0[_]an=attach[;]pl=NP\n\
# Haag	NNP	1[_]an=attach\n\
# plays	VBZ	1[_]an=juxtapose[;]pl=VP[;]nl=S\n\
# Elianti	NNP	2[_]an=attach[_]NP\n\
# .	.	1[_]an=attach\n\
# -EOS-	-EOS-	-EOS-"

l01 = "-BOS-	-BOS-	-BOS-\n\
Ms.	NNP	1[_]an=juxtapose[;]pl=VP[;]nl=S\n\
Haag	NNP	1[_]an=attach\n\
plays	VBZ	1[_]an=juxtapose[;]pl=VP[;]nl=S\n\
Elianti	NNP	2[_]an=attach[_]NP\n\
.	.	1[_]an=attach\n\
-EOS-	-EOS-	-EOS-"

# juxtaposed labels:
# (target-node-depth, action, parent-label, new-label)

gold_tree = "(S (NP (NNP Ms.) (NNP Haag)) (VP (VBZ plays) (NP (NNP Elianti))) (. .))"
pt(gold_tree)
gaps_encoder = C_JuxtaposedEncoding(separator="[_]", unary_joiner="[+]", binary=False, binary_direction="R", binary_marker="[b]")
lt = LinearizedTree.from_string(l01, mode="CONST", separator="[_]", unary_joiner="[+]")
dt = gaps_encoder.decode(lt)
print(dt)
pt(dt)

               S                  
      _________|________________   
     |              VP          | 
     |          ____|_____      |  
     NP        |          NP    | 
  ___|___      |          |     |  
NNP     NNP   VBZ        NNP    . 
 |       |     |          |     |  
Ms.     Haag plays     Elianti  . 

None
(S -NONE- (VP (NNP Ms.)))
---
(VP (NNP Ms.) (NNP Haag))
(S -NONE- (VP (NNP Ms.) (NNP Haag)))
---
None
(S -NONE- (S (VP (NNP Ms.) (NNP Haag)) (VP (VBZ plays))))
---
(VP (VBZ plays) (NP (NNP Elianti)))
(S -NONE- (S (VP (NNP Ms.) (NNP Haag)) (VP (VBZ plays) (NP (NNP Elianti)))))
---
(S (VP (NNP Ms.) (NNP Haag)) (VP (VBZ plays) (NP (NNP Elianti))) (. .))
(S -NONE- (S (VP (NNP Ms.) (NNP Haag)) (VP (VBZ plays) (NP (NNP Elianti))) (. .)))
---
-NONE-


ValueError: Tree.read(): expected '(' but got '-NONE-'
            at index 0.
                "-NONE-"
                 ^

In [ ]:
gaps_encoder = C_JuxtaposedEncoding(separator="[_]", unary_joiner="[+]", binary=False, binary_direction="R", binary_marker="[b]")

treebank_path = "/home/poli/Treebanks/const/PENN_TREEBANK/train.trees"
sample_trees = open(treebank_path).readlines()
label_set = set()
for idx, ts in enumerate(sample_trees):
    ts = ts.rstrip()
    print(ts)
    ts = C_Tree.from_string(ts)
    lc = gaps_encoder.encode(ts)
    dt = gaps_encoder.decode(lc)
    assert str(ts)==str(dt), f"Error at {idx}\n{lc}\n{ts}\n{dt}"

(S (PP (IN In) (NP (NP (DT an) (NNP Oct.) (CD 19) (NN review)) (PP (IN of) (NP (`` ``) (NP (DT The) (NN Misanthrope)) ('' '') (PP (IN at) (NP (NP (NNP Chicago) (POS 's)) (NNP Goodman) (NNP Theatre))))) (PRN (-LRB- -LRB-) (`` ``) (S (NP (JJ Revitalized) (NNS Classics)) (VP (VBP Take) (NP (DT the) (NNP Stage)) (PP (IN in) (NP (NNP Windy) (NNP City))))) (, ,) ('' '') (NP (NNP Leisure) (CC &) (NNP Arts)) (-RRB- -RRB-)))) (, ,) (NP (NP (NP (DT the) (NN role)) (PP (IN of) (NP (NNP Celimene)))) (, ,) (VP (VBN played) (PP (IN by) (NP (NNP Kim) (NNP Cattrall)))) (, ,)) (VP (VBD was) (VP (ADVP (RB mistakenly)) (VBN attributed) (PP (TO to) (NP (NNP Christina) (NNP Haag))))) (. .))
(S (NP (NNP Ms.) (NNP Haag)) (VP (VBZ plays) (NP (NNP Elianti))) (. .))
(S (NP (NNP Rolls-Royce) (NNP Motor) (NNPS Cars) (NNP Inc.)) (VP (VBD said) (SBAR (S (NP (PRP it)) (VP (VBZ expects) (S (NP (PRP$ its) (NNP U.S.) (NNS sales)) (VP (TO to) (VP (VB remain) (ADJP (JJ steady)) (PP (IN at) (NP (QP (IN about) (CD 1,200)) 

In [ ]:
tetra_encoder = C_Tetratag(separator="[_]", unary_joiner="[+]", mode="postorder", binary_marker="[b]")
# treebank_path = "/home/droca1/Treebanks/const/PENN_TREEBANK/train.trees"
# sample_trees = open(treebank_path).readlines()
sample_trees = [p15]
label_set = set()
for idx, ts in enumerate(sample_trees):
    ts = ts.rstrip()
    pb(ts)
    ts = C_Tree.from_string(ts)
    lc = tetra_encoder.encode(ts)
    print(lc)
    dt = tetra_encoder.decode(lc)
    assert str(ts)==str(dt), f"Error at {idx}\n{lc}\n{ts}\n{dt}"


                                        S                                       
             ___________________________|__________                              
            |                                      S*                           
            |                        ______________|________                     
            |                       |                       VP                  
            |                       |         ______________|____                
            NP                      |        |                  VP*             
  __________|___                    |        |           ________|___            
 |             NP*                  |        |          |            PP         
 |     _________|____               |        |          |         ___|___        
 |    |             NP*             NP       |          NP       |       NP     
 |    |          ____|____      ____|___     |      ____|___     |    ___|___    
 DT   NN        NN    

### NER2Trees


In [ ]:
from ner2trees import *

g01j = '{"tokens": ["The","binding","of","interleukin-2","(","IL-2",")","to","its","receptor","on","normal","T","cells","induces","nuclear","expression","of","nuclear","factor","kappaB","(","NF-kappaB",")",",","activation","of","the","IL-2","receptor","(","IL-2R",")","alpha","chain","gene",",","and","cell","proliferation","."],"doc_id": "MEDLINE:98281217","sent_id": "MEDLINE:98281217-1","entity_mentions": [{"start": 3,"end": 4,"entity_type": "protein","text": "interleukin-2"},{"start": 5,"end": 6,"entity_type": "protein","text": "IL-2"},{"start": 12,"end": 14,"entity_type": "cell_type","text": "T cells"},{"start": 18,"end": 21,"entity_type": "protein","text": "nuclear factor kappaB"},{"start": 22,"end": 23,"entity_type": "protein","text": "NF-kappaB"},{"start": 28,"end": 29,"entity_type": "protein","text": "IL-2"},{"start": 28,"end": 30,"entity_type": "protein","text": "IL-2 receptor"},{"start": 28,"end": 35,"entity_type": "protein","text": "IL-2 receptor ( IL-2R ) alpha chain"},{"start": 28,"end": 36,"entity_type": "DNA","text": "IL-2 receptor ( IL-2R ) alpha chain gene"}]}'

line = g01j
s = json.loads(line)
words, postags, entities = s['tokens'], ["NONE"]*len(s['tokens']), parse_entities(s['entity_mentions'], len(s['tokens']),  extractor=parse_genia) 
t = entities_to_tree(words, postags, entities)
t.clean_tree()
Tree.fromstring(str(t)).pretty_print()

                                                                                                                                                         ROOT                                                                                                                                              
  ________________________________________________________________________________________________________________________________________________________|_____________________________________________________________________________________________________________________________________________    
 |      |     |         |         |      |      |    |    |      |      |     |             |              |       |        |       |              |             |       |       |    |       |       |    |                                        DNA                    |    |    |         |        |  
 |      |     |         |         |      |      |    |    |      |      |     |             |      